<a href="https://colab.research.google.com/github/prathamchintamani/neuralnets/blob/main/MAGIC_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://archive.ics.uci.edu/dataset/267/banknote+authentication
dataset

In [555]:
#libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [556]:
import torch
from torch import nn

In [557]:
if torch.cuda.is_available():
  device = 'cpu'#gpu not working for some reason
else:
  device = 'cpu'

In [558]:
#data loading
cols = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fALpha", "fDist", "target"]
df = pd.read_csv("magic04.data",names = cols)
df["target"] = (df["target"] == "g").astype(int)
df = df.sample(frac = 1)
df = df.reset_index(drop = True)

In [559]:
#preprocessing
y = df.target
X = df.drop(["target"],axis = 1)
X_train ,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.70,random_state = 0)

In [560]:
#pddf to numpy
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [561]:
#assigning number of rows to variables
n = np.shape(X_train)[0]
m = np.shape(X_test)[0]

In [562]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [563]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [564]:
X_train = X_train.type(torch.float)
X_test = X_test.type(torch.float)
y_train = y_train.type(torch.float)
y_test = y_test.type(torch.float)

In [565]:
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

In [566]:
class neural(nn.Module):
  def __init__(self, in_size, hidden_size = 8):
    super().__init__()
    self.layer1 = nn.Linear(in_features=in_size,out_features=hidden_size)
    self.layer2 = nn.Linear(in_features=hidden_size,out_features=hidden_size)
    self.layer3 = nn.Linear(in_features=hidden_size,out_features=hidden_size)
    self.layer4 = nn.Linear(in_features=hidden_size,out_features=1)
    self.relu=nn.ReLU()
    self.sigmoid=nn.Sigmoid()
  def forward(self, X):
    return self.sigmoid(self.layer4(self.relu(self.layer3(self.relu(self.layer2(self.relu(self.layer1(X))))))))


In [567]:
model = neural(in_size = 10, hidden_size = 8).to(device)

In [568]:
print(model)

neural(
  (layer1): Linear(in_features=10, out_features=8, bias=True)
  (layer2): Linear(in_features=8, out_features=8, bias=True)
  (layer3): Linear(in_features=8, out_features=8, bias=True)
  (layer4): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [569]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [570]:
#torch.manual_seed(42)
epochs = 100000

In [571]:
for epoch in range(epochs):
    # 1. Forward pass
    y_logits = model(X_train).squeeze()
    y_pred = torch.round(y_logits) # logits -> prediction probabilities -> prediction labels

    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train) # BCEWithLogitsLoss calculates loss using logits
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model(X_test).squeeze()
      test_pred = torch.round(test_logits) # logits -> prediction probabilities -> prediction labels
      # 2. Calcuate loss and accuracy
      test_loss = loss_fn(test_logits, y_test)
      test_acc = accuracy_fn(y_true=y_test,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 1000 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.66417, Accuracy: 34.08% | Test Loss: 0.66308, Test Accuracy: 35.30%
Epoch: 1000 | Loss: 0.56962, Accuracy: 73.15% | Test Loss: 0.57446, Test Accuracy: 72.94%
Epoch: 2000 | Loss: 0.56755, Accuracy: 73.15% | Test Loss: 0.57283, Test Accuracy: 72.92%
Epoch: 3000 | Loss: 0.56675, Accuracy: 73.33% | Test Loss: 0.57221, Test Accuracy: 73.06%
Epoch: 4000 | Loss: 0.56623, Accuracy: 73.46% | Test Loss: 0.57179, Test Accuracy: 73.13%
Epoch: 5000 | Loss: 0.56582, Accuracy: 73.50% | Test Loss: 0.57146, Test Accuracy: 73.34%
Epoch: 6000 | Loss: 0.56544, Accuracy: 73.59% | Test Loss: 0.57120, Test Accuracy: 73.27%
Epoch: 7000 | Loss: 0.56509, Accuracy: 73.70% | Test Loss: 0.57093, Test Accuracy: 73.26%
Epoch: 8000 | Loss: 0.56473, Accuracy: 73.77% | Test Loss: 0.57064, Test Accuracy: 73.20%
Epoch: 9000 | Loss: 0.56435, Accuracy: 73.91% | Test Loss: 0.57032, Test Accuracy: 73.27%
Epoch: 10000 | Loss: 0.56342, Accuracy: 74.01% | Test Loss: 0.56977, Test Accuracy: 73.59%
Epoch: 11000